## Q1. Embedding the query

In [ ]:
import numpy as np
from fastembed import TextEmbedding

In [2]:
embedder = TextEmbedding(model_name='jinaai/jina-embeddings-v2-small-en')

In [5]:
query = 'I just discovered the course. Can I join now?'
q, = list(embedder.embed(query))
q.min()

np.float64(-0.11726373885183883)

## Q2. Cosine similarity with another vector

In [6]:
doc = 'Can I still join the course after the start date?'
d, = list(embedder.embed(doc))

In [7]:
q.dot(d)

np.float64(0.9008528895674548)

## Q3. Ranking by cosine

In [8]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [9]:
import pandas as pd

In [10]:
df = pd.DataFrame(documents)
df

,text,section,question,course
0,"Yes, even if you don't register, you're still ...",General course-related questions,Course - Can I still join the course after the...,data-engineering-zoomcamp
1,"Yes, we will keep all the materials after the ...",General course-related questions,Course - Can I follow the course after it fini...,data-engineering-zoomcamp
2,The purpose of this document is to capture fre...,General course-related questions,Course - When will the course start?,data-engineering-zoomcamp
3,You can start by installing and setting up all...,General course-related questions,Course - What can I do before the course starts?,data-engineering-zoomcamp
4,Star the repo! Share it with friends if you fi...,General course-related questions,How can we contribute to the course?,data-engineering-zoomcamp


In [11]:
v_text = list(embedder.embed(df.text))
v_text = np.array(v_text)

In [12]:
similarity = v_text.dot(q)
similarity

array([0.76296847, 0.81823782, 0.80853974, 0.7133079 , 0.73044992])

In [13]:
similarity.argmax()

np.int64(1)

## Q4. Ranking by cosine, version two

In [14]:
v_text = list(embedder.embed(df.question + ' ' + df.text))
v_text = np.array(v_text)

In [15]:
similarity = v_text.dot(q)
similarity

array([0.85145432, 0.84365942, 0.8408287 , 0.7755158 , 0.80860078])

In [16]:
similarity.argmax()

np.int64(0)

## Q5. Selecting the embedding model

In [17]:
models = TextEmbedding.list_supported_models()
df_models = pd.DataFrame(models)

In [18]:
# the smallest dimensionality models in fastembed
df_models[df_models.dim == df_models.dim.min()]

,model,sources,model_file,description,license,size_in_GB,additional_files,dim,tasks
3,BAAI/bge-small-en,"{'hf': 'Qdrant/bge-small-en', 'url': 'https://...",model_optimized.onnx,"Text embeddings, Unimodal (text), English, 512...",mit,0.130,[],384,{}
4,BAAI/bge-small-en-v1.5,"{'hf': 'qdrant/bge-small-en-v1.5-onnx-q', 'url...",model_optimized.onnx,"Text embeddings, Unimodal (text), English, 512...",mit,0.067,[],384,{}
7,snowflake/snowflake-arctic-embed-xs,"{'hf': 'snowflake/snowflake-arctic-embed-xs', ...",onnx/model.onnx,"Text embeddings, Unimodal (text), English, 512...",apache-2.0,0.090,[],384,{}
8,snowflake/snowflake-arctic-embed-s,"{'hf': 'snowflake/snowflake-arctic-embed-s', '...",onnx/model.onnx,"Text embeddings, Unimodal (text), English, 512...",apache-2.0,0.130,[],384,{}
14,sentence-transformers/all-MiniLM-L6-v2,"{'hf': 'qdrant/all-MiniLM-L6-v2-onnx', 'url': ...",model.onnx,"Text embeddings, Unimodal (text), English, 256...",apache-2.0,0.090,[],384,{}
26,sentence-transformers/paraphrase-multilingual-...,{'hf': 'qdrant/paraphrase-multilingual-MiniLM-...,model_optimized.onnx,"Text embeddings, Unimodal (text), Multilingual...",apache-2.0,0.220,[],384,{}


## Q6. Indexing with qdrant

In [19]:
# FAQ records from ml zoomcamp:

import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [20]:
!docker run -d --name qdrant -p 6333:6333 qdrant/qdrant

7b28731817e6b56d1176360561b4b5125c72e9b8d3ee1f6dae330801bf8b5326


Unable to find image 'qdrant/qdrant:latest' locally
latest: Pulling from qdrant/qdrant
59e22667830b: Pulling fs layer
4f4fb700ef54: Pulling fs layer
4f4fb700ef54: Pulling fs layer
4c5e863cd1c7: Pulling fs layer
4f4fb700ef54: Pulling fs layer
be967389c82d: Pulling fs layer
4f4fb700ef54: Pulling fs layer
4f4fb700ef54: Pulling fs layer
34d10ac9af97: Pulling fs layer
642422c7a320: Pulling fs layer
4f4fb700ef54: Pulling fs layer
4f4fb700ef54: Pulling fs layer
bdeb1045e503: Pulling fs layer
3e5bd9a6d878: Pulling fs layer
4f4fb700ef54: Already exists
34d10ac9af97: Download complete
3e5bd9a6d878: Download complete
642422c7a320: Download complete
59e22667830b: Download complete
59e22667830b: Pull complete
be967389c82d: Download complete
4c5e863cd1c7: Download complete
4c5e863cd1c7: Pull complete
4f4fb700ef54: Pull complete
be967389c82d: Pull complete
bdeb1045e503: Download complete
34d10ac9af97: Pull complete
3e5bd9a6d878: Pull complete
bdeb1045e503: Pull complete
642422c7a320: Pull complete
Di

In [21]:
from qdrant_client import QdrantClient, models
qd_client = QdrantClient("http://localhost:6333")

In [22]:
# model
EMBEDDING_DIMENSIONALITY = 384
model = "BAAI/bge-small-en"
collection_name = "llmzoomcamp-homework"

In [23]:
qd_client.delete_collection(collection_name=collection_name)

qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [24]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

qd_client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]c:\Users\alfon\Documents\courses\llm-zoomcamp\hw\week2\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\alfon\AppData\Local\Temp\fastembed_cache\models--Qdrant--bge-small-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Fetching 5 files:  40

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [26]:
question = 'I just discovered the course. Can I join now?'

query_points = qd_client.query_points(
    collection_name=collection_name,
    query=models.Document(
        text=question,
        model=model 
    ),
    limit=5,
    with_payload=True
)

In [27]:
query_points.points[0]

ScoredPoint(id=14, version=0, score=0.8703172, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)